# RNNでArrayの足し算を作ってみる
RNNは時系列データを扱うのに有用なNNである。

特徴としては、中間層に自己へのフィードバックループがあり、それにより前のステートからの影響を保持することが出来る。

今回のタスクはそれほどRNNで解く必要はないが、線形回帰などの簡単なタスクがRNNにあまりなく、自然言語処理などをし始めるとなにをしているのかがわからなくなる。

と思っていたら同じことを思っている人がおられて、参考にさせてもらった。 http://qiita.com/TomokIshii/items/01c2171f4def1a128fd3 （こちらはTheanoでの実装です）

今回のタスクの詳細はURLに譲り、chainerで実装していく。

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from sklearn.cross_validation import train_test_split

/Users/kasajei/.virtualenvs/ml3/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
x = 0.1
data = []
for _ in range(0, 10000):
    data.append([np.around(np.random.rand(10)).astype(int)])
print(data[:5])
data = np.asarray(data, dtype=np.float32)
y = np.sum(data, axis=2)
print(y[:5])

[[array([0, 0, 1, 0, 0, 1, 1, 0, 1, 0])], [array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0])], [array([0, 1, 0, 1, 1, 1, 1, 0, 0, 0])], [array([1, 1, 0, 0, 0, 0, 1, 1, 1, 0])], [array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])]]
[[ 4.]
 [ 3.]
 [ 5.]
 [ 5.]
 [ 1.]]


In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(data, y, test_size=0.1, random_state=0)
print(len(X_test),len(Y_test))

1000 1000


In [6]:
class RNN(Chain):
    def __init__(self):
        super(RNN, self).__init__(
            i = L.Linear(1,1), # 入力→隠れ
            h = L.Linear(1,1), # 隠れ→隠れ
            o =  L.Linear(1,1) # 隠れ→出力
        )        
        
    def __call__(self, x_array, y):
        h = Variable(np.zeros((len(x_array.data),  1), dtype=np.float32), volatile=x_array.volatile) # 隠れ層の初期値
        o = Variable(np.zeros((len(x_array.data),  1), dtype=np.float32), volatile=x_array.volatile) # 隠れ層の初期値
        for x in x_array.data.T:
            i = self.i(Variable(x.T, volatile=x_array.volatile))
            h = i + self.h(h)

        o = self.o(h)
        self.y = o
        self.loss = F.mean_squared_error(o,y)
        return self.loss

In [7]:
model = RNN()
optimizer = optimizers.Adam()
optimizer.setup(model)

batch_size = int(len(X_train) * 0.1)
epoch = 10001
model.zerograds()
for step in range(epoch):
    batch_index = np.random.randint(len(X_train)-batch_size)
    batch_x =Variable(X_train[batch_index:batch_index+batch_size])
    batch_y = Variable(Y_train[batch_index:batch_index+batch_size])
    optimizer.update(model, batch_x, batch_y)
    if step % ((epoch-1)/10) == 0:
        print(step,"ステップ→",model.loss.data) 

0 ステップ→ 39.934722900390625
1000 ステップ→ 4.527037143707275
2000 ステップ→ 2.7627742290496826
3000 ステップ→ 2.3172922134399414
4000 ステップ→ 2.0130116939544678
5000 ステップ→ 1.7121180295944214
6000 ステップ→ 9.771873010322452e-06
7000 ステップ→ 5.189105048430953e-11
8000 ステップ→ 1.2565976037492987e-11
9000 ステップ→ 5.394258824342746e-12
10000 ステップ→ 9.641393586279356e-13


In [8]:
x_variable = Variable(X_test, volatile=True)
y_variable = Variable(Y_test, volatile=True)
# model.reset_state()
model(x_variable,y_variable)
print(model.loss.data)
print(Y_test[:5])
print(model.y.data[:5])

1.9205401158295388e-12
[[ 4.]
 [ 8.]
 [ 6.]
 [ 7.]
 [ 4.]]
[[ 3.99999714]
 [ 7.99999905]
 [ 5.99999857]
 [ 6.99999857]
 [ 3.99999833]]


# LSTM
中間層をchainerが用意してくれているLSTMにしてみる

In [9]:
class LSTM(Chain):
    def __init__(self):
        super(LSTM, self).__init__(
            i = L.Linear(1,1), # 入力→隠れ
            h = L.LSTM(1,1), # 隠れ→隠れ
            o =  L.Linear(1,1) # 隠れ→出力
        )        
            
    def __call__(self, x_array, y):
        o = Variable(np.zeros((len(x_array.data),  1), dtype=np.float32), volatile=x_array.volatile) # 隠れ層の初期値
        for x in x_array.data.T:
            i = self.i(Variable(x.T, volatile=x_array.volatile))
            h = self.h(i)
            
        o = self.o(h)
        self.h.reset_state()
        self.y = o
        self.loss = F.mean_squared_error(o,y)
        return self.loss

In [10]:
model = LSTM()
optimizer = optimizers.Adam()
optimizer.setup(model)

batch_size = int(len(X_train) * 0.1)
epoch = 10001
model.zerograds()
for step in range(epoch):
    batch_index = np.random.randint(len(X_train)-batch_size)
    batch_x =Variable(X_train[batch_index:batch_index+batch_size])
    batch_y = Variable(Y_train[batch_index:batch_index+batch_size])
    optimizer.update(model, batch_x, batch_y)
    if step % ((epoch-1)/10) == 0:
        print(step,"ステップ→",model.loss.data) 

0 ステップ→ 27.03696632385254
1000 ステップ→ 9.822260856628418
2000 ステップ→ 4.30447244644165
3000 ステップ→ 2.6470730304718018
4000 ステップ→ 2.448301076889038
5000 ステップ→ 1.2512857913970947
6000 ステップ→ 0.18731147050857544
7000 ステップ→ 0.12418320029973984
8000 ステップ→ 0.07552158832550049
9000 ステップ→ 0.04857984930276871
10000 ステップ→ 0.026926083490252495


In [11]:
x_variable = Variable(X_test, volatile=True)
y_variable = Variable(Y_test, volatile=True)
# model.reset_state()
model(x_variable,y_variable)
print(model.loss.data)
print(Y_test[:5])
print(model.y.data[:5])

0.02813943289220333
[[ 4.]
 [ 8.]
 [ 6.]
 [ 7.]
 [ 4.]]
[[ 3.93993902]
 [ 7.64936924]
 [ 6.14246273]
 [ 6.91297626]
 [ 3.92608857]]
